In [ ]:
import requests
from collections import Counter
import nltk
nltk.download('punkt')
import pandas as pd
from nltk.tokenize import word_tokenize
import numpy as np


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
file_path = '/content/text_dataset_ext_val.csv'
df = pd.read_csv(file_path, sep=',')
#print(df.iloc[0:10])
print(df.head())
df

                        file_name  \
0  Baycrest10285trimmed_plain.txt   
1  Baycrest11014trimmed_plain.txt   
2  Baycrest12756trimmed_plain.txt   
3  Baycrest12828trimmed_plain.txt   
4  Baycrest12829trimmed_plain.txt   

                                                text  classifier  
0  kind of a quiet person to start with and now I...           1  
1  very happy with how they what I'm doing okay y...           1  
2  I would say when I'm married. Great. A typical...           1  
3  It's not good.And everybody I see say; because...           1  
4  Well; my first wife had her baby and it was 17...           1  


,file_name,text,classifier
0,Baycrest10285trimmed_plain.txt,kind of a quiet person to start with and now I...,1
1,Baycrest11014trimmed_plain.txt,very happy with how they what I'm doing okay y...,1
2,Baycrest12756trimmed_plain.txt,I would say when I'm married. Great. A typical...,1
3,Baycrest12828trimmed_plain.txt,It's not good.And everybody I see say; because...,1
4,Baycrest12829trimmed_plain.txt,Well; my first wife had her baby and it was 17...,1
5,Baycrest13074trimmed_plain.txt,It has improved. I am trying to concentrate mo...,1
6,Baycrest13156trimmed_plain.txt,Every day I take it and; you know; practice it...,1
7,Baycrest13187trimmed_plain.txt,We were born in the farm.And cows; horses; and...,1
8,Baycrest13188trimmed_plain.txt,the 40 for the and the process here and of Rus...,1
9,Baycrest13196trimmed_plain.txt,It's not as good as one. It was. I can remembe...,1


In [ ]:
#syntactic features
def check_text_with_languagetool(text):
    url = "https://api.languagetoolplus.com/v2/check"
    payload = {
        'text': text,
        'language': 'en-US'
    }
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    response = requests.post(url, data=payload, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return "Error in LanguageTool API call"

def get_grammar_error_count(text):
    grammar_results = check_text_with_languagetool(text)
    if isinstance(grammar_results, dict):
        return len(grammar_results.get('matches', []))
    else:
        return 0  # Assume no errors if there's an issue with the API call

def count_repeated_words(text):
    sentences = text.split('.')
    total_repeats = 0
    for sentence in sentences:
        words = sentence.lower().split()
        word_counts = Counter(words)
        total_repeats += sum(count - 1 for count in word_counts.values() if count > 1)
    return total_repeats

In [ ]:
#lexical features
def vocabulary_length(text):
  words = word_tokenize(text.lower())
  word_counts = Counter(words)
  return len(word_counts)

def lexical_diversity(text):
    words = word_tokenize(text.lower())
    text_length = len(words)
    word_counts = Counter(words)
    vl = len(word_counts)
    if vl == 0:
        return 0
    return text_length / vl

In [ ]:
#semantic features
from transformers import RobertaModel, RobertaTokenizer
import torch

In [ ]:
def get_roberta_embeddings(texts, use_cls_token=False):
    # Initialize tokenizer and model
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    model = RobertaModel.from_pretrained('roberta-base')

    # Tokenize input texts
    encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt", max_length=128)

    # Forward pass, get hidden states
    with torch.no_grad():
        outputs = model(**encodings)

    if use_cls_token:
        # Use the embedding of the CLS token (first token)
        embeddings = outputs.last_hidden_state[:, 0, :]
    else:
        # Average the embeddings of all tokens
        embeddings = outputs.last_hidden_state.mean(dim=1)

    # Calculate the L2 norm of each embedding vector
    norms = torch.norm(embeddings, p=2, dim=1).numpy()  # L2 norm along the feature dimension
    return norms

In [ ]:
embeddings = get_roberta_embeddings(df['text'].tolist(), use_cls_token=False)
#print(embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
df['repeated_word_count'] = df['text'].apply(count_repeated_words)
df['mistakes'] = df['text'].apply(get_grammar_error_count)
df['vocabulary_length'] = df['text'].apply(vocabulary_length)
df['lexical_diversity'] = df['text'].apply(lexical_diversity)


In [ ]:
df['embeddings'] = embeddings
print(df.head())


                        file_name  \
0  Baycrest10285trimmed_plain.txt   
1  Baycrest11014trimmed_plain.txt   
2  Baycrest12756trimmed_plain.txt   
3  Baycrest12828trimmed_plain.txt   
4  Baycrest12829trimmed_plain.txt   

                                                text  classifier  \
0  kind of a quiet person to start with and now I...           1   
1  very happy with how they what I'm doing okay y...           1   
2  I would say when I'm married. Great. A typical...           1   
3  It's not good.And everybody I see say; because...           1   
4  Well; my first wife had her baby and it was 17...           1   

   repeated_word_count  mistakes  vocabulary_length  lexical_diversity  \
0                  250         0                317           2.561514   
1                  209         0                228           2.745614   
2                   36         0                174           2.896552   
3                   48         0                239           2.673640  

In [ ]:
df.head()

,file_name,text,classifier,repeated_word_count,mistakes,vocabulary_length,lexical_diversity,embeddings
0,Baycrest10285trimmed_plain.txt,kind of a quiet person to start with and now I...,1,250,0,317,2.561514,12.187510
1,Baycrest11014trimmed_plain.txt,very happy with how they what I'm doing okay y...,1,209,0,228,2.745614,12.529081
2,Baycrest12756trimmed_plain.txt,I would say when I'm married. Great. A typical...,1,36,0,174,2.896552,12.447489
3,Baycrest12828trimmed_plain.txt,It's not good.And everybody I see say; because...,1,48,0,239,2.673640,11.970352
4,Baycrest12829trimmed_plain.txt,Well; my first wife had her baby and it was 17...,1,29,17,123,1.967480,11.981928


In [ ]:
df.to_csv(path_or_buf='/content/text_features_ext_val.csv', sep=';')

In [ ]:
# now, we merge the text features dataframe "df" with the audio features df "df_audio"

file_path = '/content/audio_features_ext_valid.csv'
df_audio = pd.read_csv(file_path, sep=',')
df_audio.rename(columns={"file_name": "aud_file_name"}, inplace = True)
df_audio = df_audio.drop(['classifier'], axis=1)

df_all = pd.concat([df, df_audio], axis=1) # careful: every time it is run, it will concat a copy of df_emb to df_all
df_all.rename(columns={"file_name": "txt_file_name"}, inplace = True)

df_all.to_csv(path_or_buf='/content/all_features_ext_val.csv', sep=';')

df_all.head()

,txt_file_name,text,classifier,repeated_word_count,mistakes,vocabulary_length,lexical_diversity,embeddings,aud_file_name,@attribute F0semitoneFrom27.5Hz_sma3nz_amean numeric,...,@attribute slopeUV500-1500_sma3nz_amean numeric,@attribute spectralFluxUV_sma3nz_amean numeric,@attribute loudnessPeaksPerSec numeric,@attribute VoicedSegmentsPerSec numeric,@attribute MeanVoicedSegmentLengthSec numeric,@attribute StddevVoicedSegmentLengthSec numeric,@attribute MeanUnvoicedSegmentLength numeric,@attribute StddevUnvoicedSegmentLength numeric,@attribute equivalentSoundLevel_dBp numeric,@attribute class numeric
0,Baycrest10285trimmed_plain.txt,kind of a quiet person to start with and now I...,1,250,0,317,2.561514,12.187510,Baycrest10285trimmed_egemaps.csv,24.9,...,0.00145,0.0402,0.472,1.24,0.367,0.294,0.4310,0.6000,-34.6,1
1,Baycrest11014trimmed_plain.txt,very happy with how they what I'm doing okay y...,1,209,0,228,2.745614,12.529081,Baycrest11014trimmed_egemaps.csv,20.9,...,-0.01690,0.3390,1.520,1.63,0.562,0.656,0.0471,0.0464,-29.1,1
2,Baycrest12756trimmed_plain.txt,I would say when I'm married. Great. A typical...,1,36,0,174,2.896552,12.447489,Baycrest12756trimmed_egemaps.csv,20.4,...,-0.01650,0.5160,1.480,2.58,0.326,0.436,0.0549,0.0457,-18.4,1
3,Baycrest12828trimmed_plain.txt,It's not good.And everybody I see say; because...,1,48,0,239,2.673640,11.970352,Baycrest12828trimmed_egemaps.csv,27.0,...,-0.01720,0.6370,1.660,1.37,0.680,0.790,0.0448,0.0408,-18.8,1
4,Baycrest12829trimmed_plain.txt,Well; my first wife had her baby and it was 17...,1,29,17,123,1.967480,11.981928,Baycrest12829trimmed_egemaps.csv,20.5,...,-0.01480,1.0100,1.220,2.70,0.305,0.386,0.0588,0.0531,-18.1,1
